#### 개요
학습용 또는 온라인 시험용을 생성한 샘플 데이터를 전처리에 사용할 수 있도록 공사비 단위로 하나의 레코드로 취합하는 역할을 수행
* 온라인/배치 부분이 구분되어 구현되 있음

##### 진행사항
* STEP01: 각 데이터 셋에서 학습대상 레코드와 컬럼 추출
  * 이 부분은 온라인과 배치 부분이 상반되기 때문에 별도로 구현함
* STEP02: 공사비별 전주/전선/인입선 갯 수 계산 및 체크
* STEP03: 하나의 데이터프레임으로 취합

In [1]:
import json
import pandas as pd

import aidd.sys.config as cfg
from aidd.batch.data_manager import get_provide_data

In [2]:
pdata = get_provide_data()

[6835b81b9c48][2024-03-13 18:00:05.757080] 제공받은 데이터 불러오기 시작
[6835b81b9c48][2024-03-13 18:00:32.894634]   읽은 데이터: CONS, 데이터 크기: (19052, 143), 처리시간: 0:00:27.137447
[6835b81b9c48][2024-03-13 18:00:57.817161]   읽은 데이터: POLE, 데이터 크기: (38533, 63), 처리시간: 0:00:24.922387
[6835b81b9c48][2024-03-13 18:01:28.818265]   읽은 데이터: LINE, 데이터 크기: (40019, 77), 처리시간: 0:00:31.000967
[6835b81b9c48][2024-03-13 18:01:41.490202]   읽은 데이터: SL, 데이터 크기: (22632, 57), 처리시간: 0:00:12.671806
[6835b81b9c48][2024-03-13 18:01:41.490293] 제공받은 데이터 불러오기 종료, 최종 처리시간: 0:01:35.733221


#### STEP01: 각 데이터 셋에서 학습대상 레코드와 컬럼 추출

##### 배치 작업

In [3]:
# 모델에 사용할 데이터 딕셔너리
batch_dict = {}

# 공사비 데이터 학습대상 레코드/컬럼 추출
key = 'CONS'
df = pdata[key]
# (전주/전선 수를 제외한) 공사비 데이터 부분에서 학습 대상 레코드 조건
# * 접수종류명(ACC_TYPE_NAME), 계약전력(CONT_CAP),
# * 공사형태코드(CONS_TYPE_CD), 총공사비(TOTAL_CONS_COST)
modeling_records = \
    (df.ACC_TYPE_NAME == cfg.COND_ACC_TYPE_NAME) & \
    (df.CONT_CAP < cfg.COND_MAX_CONT_CAP) & \
    (df.CONS_TYPE_CD == cfg.COND_CONS_TYPE_CD) & \
    (df.TOTAL_CONS_COST < cfg.COND_MAX_TOTAL_CONS_COST)
df = df[modeling_records].reset_index(drop=True)
cons_df = df[cfg.MODELING_COLS[key]]
batch_dict[key] = cons_df

# 전주/전선/인입선 데이터 학습대상 레코드/컬럼 추출
# * 레크도는 공사비에 있는 공사번호를 대상으로 함
for key in cfg.DATA_TYPE[1:]:
    df = pdata[key]
    df = df[df.CONS_ID.isin(cons_df.CONS_ID)]
    batch_dict[key] = df[cfg.MODELING_COLS[key]]
    
# (참조용)데이터 셋 별 공사대상 레코드/컬럼 크기 확인
for key in cfg.DATA_TYPE:
    print(f'{key}: {batch_dict[key].shape}')

CONS: (15335, 8)
POLE: (27896, 6)
LINE: (30348, 11)
SL: (17034, 6)


##### 온라인 작업

In [4]:
def get_online_sample(data):
    """온라인 서비스 샘플 데이터 생성 함수
    * 온라인 서비스나 온라인 서비스를 위한 전처리 프로세스 테스트를 위한
      샘플 데이터를 생성하는 함수로, 대상 데이터는 `config`에 지정되어 있음
    * 실제 온라인 작업에선 사용하지 않음
    Args:
        data (dict): 학습용으로 제공받은 전체 데이터
            - `key: df`로 구성된 딕셔너리로 제공됨
    Returns:
        jsons (dict): Online으로 받을 예정인 JSON 데이터의 딕셔너리
    """
    # 전체 데이터에서 샘플 데이터만 추출한 데이터프레임 딕셔너리
    jsons = {}
    for key in cfg.DATA_TYPE:
        df = data[key]
        df = df[df.CONS_ID.isin(cfg.CHECK_CONS_IDS)]
        # 공사비 데이터셋에 있는 총공사비를 제거하는 코드(3줄이나 소요됨;;)
        modeling_cols = cfg.MODELING_COLS[key]
        if 'TOTAL_CONS_COST' in modeling_cols:
            modeling_cols.remove('TOTAL_CONS_COST')
        jsons[key] = df[modeling_cols].to_json(orient='records')
        
    return jsons

In [5]:
json_dict = get_online_sample(data=pdata)

# 이 부분은 추후에 수정할 사항이 많아 보임
# RESTful로 입력받은 json데이터를 데이터프레임으로 변환

# 입력된 데이터를 데이터프레임으로 변환한 딕셔너리
# 모델에 사용할 데이터 딕셔너리
online_dict = {}
for key in json_dict.keys():
    online_dict[key] = pd.read_json(json_dict[key])
# 총공사비가 없는 경우에는 총공사비 컬럼을 0으로 추가
# 예측해야할 값이긴 하지만 온라인/배치 전처리를 같은 함수로 하기 위해 
# 컬럼만 추가함
if 'TOTAL_CONS_COST' not in online_dict['CONS'].keys():
    online_dict['CONS']['TOTAL_CONS_COST'] = 0
    
# (참조용)데이터 셋 별 공사대상 레코드/컬럼 크기 확인
for key in cfg.DATA_TYPE:
    print(f'{key}: {online_dict[key].shape}')

CONS: (3, 8)
POLE: (9, 6)
LINE: (9, 11)
SL: (3, 6)


#### STEP02~3: 하나의 클래스로 처리

In [ ]:
class MergingSourceData():
    """" 4개의 데이터프레임 취합
        CONS, POLE, LINE, SL 데이터를 하나로 묶는 작업을 하며,
        CONS의 공사번호별로 POLE, LINE, SL의 카운트는 계산하지만
        컬럼 추가작업이나 스케일링 등은 여기서 처리하지 않음
    """
    def __init__(self, pdata=None, is_batch=True):
        self.pdata = pdata
        self.is_batch = is_batch
        self.merge_df = None
        self._run()
        return self.merge_df
    
    def _run(self):
        self._compute_and_check_facilities_count()
        
    def _compute_and_check_facilites_count(self):
        pass